### Using FHIR to find files for genomic analysis 

We are imagining here we are a genomic researcher who wants to query on phenotypic attributes of patients within a study and do a compute. We know we can get DRS ids out of the Kids First data - and we know how ot do a compute on those files onve we have a DRS id (see other notebooks for how). So we have a SMART goal to shoot for.

At this point we can't see what study a Patient is part of. The relationship to a study is a relationship that belongs to ResearchSubject not Patient.

Can we find what Research Studies exist?

In [24]:
import sys
import os
import json
import requests

#FHIR_SERVER = 'https://kf-api-fhir-service.kidsfirstdrc.org'
FHIR_SERVER = 'https://ncpi-api-fhir-service-dev.kidsfirstdrc.org'

# Optional: Turn off SSL verification. Useful when dealing with a corporate proxy with self-signed certificates.
# This should be set to True unless you actually see certificate errors.
VERIFY_SSL = False

if not VERIFY_SSL:
    requests.packages.urllib3.disable_warnings()



# Kids First uses cookie-based authentication
# Get my locally saved cookie
full_cookie_path = os.path.expanduser('~/.keys/ncpi_fhir_cookie.json')
with open(full_cookie_path) as f:
        cookies = json.load(f)

# We make a requests.Session to ensure consistent headers/cookie across all the requests we make
sess = requests.Session()
sess.headers.update({'Accept': 'application/fhir+json'})
sess.headers.update(cookies)
sess.verify = VERIFY_SSL


# Test out the cookie by querying the server metadata
r = sess.get(f"{FHIR_SERVER}/metadata")

if "<!DOCTYPE html>" in r.text:
    sys.stderr.write('ERROR: Could not authenticate with Kids First. The cookie may need to be updated')

In [25]:
r = sess.get(f"{FHIR_SERVER}/ResearchStudy")
study_bundle = r.json()

# In the bundle obtained total is not present
#print(f"Number of matches: {patient_bundle['total']}")
print(f"Number of Studies included in Bundle: {len(study_bundle['entry'])}")
studies = [entry['resource'] for entry in study_bundle['entry']]

for s in studies:

    print (s['identifier'][1]['value'])
    print (s['title'])
    print(json.dumps(s, indent=3))
    print('_'*40)

Number of Studies included in Bundle: 7
ResearchStudy|phs001987
Genome-wide Sequencing to Identify the Genes Responsible for Enchondromatoses and Related Malignant Tumors
{
   "resourceType": "ResearchStudy",
   "id": "539879",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2021-04-28T23:43:17.331+00:00",
      "source": "#3dDKp2NrF3FLtKGS",
      "profile": [
         "http://hl7.org/fhir/StructureDefinition/ResearchStudy"
      ]
   },
   "identifier": [
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/studies/",
         "value": "SD_7NQ9151J"
      },
      {
         "system": "urn:kids-first:unique-string",
         "value": "ResearchStudy|phs001987"
      },
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/studies/",
         "value": "?external_id=phs001987&version=v1.p1"
      }
   ],
   "title": "Genome-wide Sequencing to Identify the Genes Responsible for Enchondromatoses and Related Malignant Tumors",
   "status": "c

Query for the Familial Leukemia study as I have access to that

In [27]:
r = sess.get(f"{FHIR_SERVER}/ResearchStudy?_id=577137")
study_bundle = r.json()

print(f"Number of Studies included in Bundle: {len(study_bundle['entry'])}")

# Create list of just the Patient Resources in the Bundle
studies = [entry['resource'] for entry in study_bundle['entry']]
print(json.dumps(studies[0], indent=3))

Number of Studies included in Bundle: 1
{
   "resourceType": "ResearchStudy",
   "id": "577137",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2021-04-28T23:57:32.722+00:00",
      "source": "#pP70Q2ef3A0Tsaev",
      "profile": [
         "http://hl7.org/fhir/StructureDefinition/ResearchStudy"
      ]
   },
   "identifier": [
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/studies/",
         "value": "SD_W0V965XZ"
      },
      {
         "system": "urn:kids-first:unique-string",
         "value": "ResearchStudy|phs001738"
      },
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/studies/",
         "value": "?external_id=phs001738&version=v1.p1"
      }
   ],
   "title": "Genomic Analysis of Familial Leukemia",
   "status": "completed",
   "principalInvestigator": {
      "reference": "PractitionerRole/575854"
   }
}


ResearchSubject is being used. What do the resources look like.

In [30]:
r = sess.get(f"{FHIR_SERVER}/ResearchSubject")
subject_bundle = r.json()

# In the bundle obtained total is not present
#print(f"Number of matches: {subject_bundle['total']}")
print(f"Number of Subjects included in Bundle: {len(subject_bundle['entry'])}")
subjects = [entry['resource'] for entry in subject_bundle['entry']]

for sub in subjects:
    print(json.dumps(sub))
    print('_____________________')

Number of Subjects included in Bundle: 50
{"resourceType": "ResearchSubject", "id": "539892", "meta": {"versionId": "1", "lastUpdated": "2021-04-28T23:43:18.486+00:00", "source": "#l4Q9C6v8ajPLXNuT", "profile": ["http://hl7.org/fhir/StructureDefinition/ResearchSubject"]}, "identifier": [{"system": "https://kf-api-dataservice.kidsfirstdrc.org/participants/", "value": "PT_09TAPX9A"}, {"system": "https://kf-api-dataservice.kidsfirstdrc.org/participants/", "value": "?study_id=SD_7NQ9151J&external_id=BH12383_3"}, {"system": "urn:kids-first:unique-string", "value": "ResearchSubject|SD_7NQ9151J|BH12383_3"}], "status": "off-study", "study": {"reference": "ResearchStudy/539879"}, "individual": {"reference": "Patient/539322"}}
_____________________
{"resourceType": "ResearchSubject", "id": "539898", "meta": {"versionId": "1", "lastUpdated": "2021-04-28T23:43:18.537+00:00", "source": "#SmT2If8pIBe8TmHF", "profile": ["http://hl7.org/fhir/StructureDefinition/ResearchSubject"]}, "identifier": [{"sys

Getting ourselves a list of the resources actually used in Kids First.

In [33]:
populatedRes = {'ResearchSubject': 11529,
 'DocumentReference': 81538,
 'PractitionerRole': 3,
 'Practitioner': 4,
 'Group': 1614,
 'Organization': 22,
 'SearchParameter': 1635,
 'Task': 3212,
 'ResearchStudy': 7,
 'Specimen': 55715,
 'StructureDefinition': 664,
 'OperationDefinition': 46,
 'ValueSet': 1329,
 'CodeSystem': 1070,
 'Observation': 7957,
 'DiagnosticReport': 5,
 'Condition': 120405,
 'CompartmentDefinition': 5,
 'Patient': 11538}

Let's explore some of them. 
Condition would be useful if we eanted to find subjects with a particular disease.

In [32]:
response = sess.get(f"{FHIR_SERVER}/Condition")
cond_bundle = response.json()

# In the bundle obtained total is not present
#print(f"Number of matches: {subject_bundle['total']}")
print(f"Number of Condition included in Bundle: {len(cond_bundle['entry'])}")
conditions = [entry['resource'] for entry in cond_bundle['entry']]

for condition in conditions:
    print(json.dumps(condition))
    print('_____________________')

Number of Condition included in Bundle: 50
{"resourceType": "Condition", "id": "705499", "meta": {"versionId": "1", "lastUpdated": "2021-06-16T02:11:43.410+00:00", "source": "#VF4M7r114ywKvjTL", "profile": ["https://ncpi-fhir.github.io/ncpi-fhir-ig/StructureDefinition/disease"]}, "identifier": [{"system": "https://kf-api-dataservice.kidsfirstdrc.org/diagnoses/", "value": "DG_0SNHKFGV"}], "clinicalStatus": {"coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-clinical", "code": "active", "display": "Active"}], "text": "Active"}, "verificationStatus": {"coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-ver-status", "code": "confirmed", "display": "Confirmed"}], "text": "True"}, "category": [{"coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-category", "code": "encounter-diagnosis", "display": "Encounter Diagnosis"}]}], "code": {"coding": [{"system": "http://purl.obolibrary.org/obo/mondo.owl", "code": "MONDO:0024879"}], "text": "M

### Getting the BAM files
It turns out the BAM files have implemented as Documents in FHIR. Their details can be accessed via the DocumentReference resource.

We filter on documents which use a profile defined for NCPI DRS references

In [43]:
response = sess.get(f'{FHIR_SERVER}/DocumentReference?_profile=http://fhir.ncpi-project-forge.io/StructureDefinition/ncpi-drs-document-reference')
drs_id_bundle = response.json()

# In the bundle obtained total is not present
#print(f"Number of matches: {doc_bundle['total']}")
print(f"Number of DRS files included in Bundle: {len(drs_id_bundle['entry'])}")
drs_ids = [entry['resource'] for entry in drs_id_bundle['entry']]

for drs_id in drs_ids:
    print(json.dumps(drs_id))
    print('_'*50)

Number of DRS files included in Bundle: 50
{"resourceType": "DocumentReference", "id": "1000G-high-coverage-2019-Sa-ERS4367868-cram", "meta": {"versionId": "1", "lastUpdated": "2021-01-04T20:02:14.883+00:00", "source": "#c2lcTJ66WxhLoG7z", "profile": ["http://fhir.ncpi-project-forge.io/StructureDefinition/ncpi-drs-document-reference"]}, "identifier": [{"system": "https://anvil.terra.bio/#workspaces/anvil-datastorage/1000G-high-coverage-2019", "value": "1000G-high-coverage-2019-Sa-ERS4367868-cram"}, {"system": "urn:ncpi:unique-string", "value": "1000G-high-coverage-2019-Sa-ERS4367868-cram"}], "status": "current", "subject": {"reference": "Patient/P.1000G-high-coverage-2019-Su-ERS4367868"}, "content": [{"attachment": {"url": "drs://dg.ANV0:dg.ANV0/5201d2e1-0524-488a-9d02-1d3783915f63"}, "format": {"display": "cram"}}]}
__________________________________________________
{"resourceType": "DocumentReference", "id": "1000G-high-coverage-2019-Sa-ERS4367796-gVCF-TBI", "meta": {"versionId": "1"

How would we find all the Document References for a patient?

In [68]:
response = sess.get(f"{FHIR_SERVER}/DocumentReference?subject=697194")
bundle = response.json()
documents = [entry['resource'] for entry in bundle['entry']]
print("# of documents:{}".format(len(documents)))

# of documents:38


Create a dataframe for the files

In [61]:
import pandas as pd
myDocs = []
for d in documents:
    kfid = [did for did in d['identifier'] if did['system']=='urn:kids-first:unique-string']
    myDocs.append({
        "kfid":kfid[0]['value'],
    "type":d['type']['text'],
    "format":d['content'][0]['format']['display']
    #"drs":djson['content'][0]['attachment']['url']
                  }
    )

docsDF = pd.DataFrame(myDocs)
docsDF = docsDF.sort_values(["type", "format"])
from IPython.display import display, HTML
docsDF.style.set_properties(subset=['kfid'], **{'width': '12px'})
display (docsDF)

,kfid,type,format
0,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,bam
5,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,bam
9,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,bam
19,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,bam
23,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,bam
7,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,cram
24,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,cram
29,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,cram
30,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads,cram
6,DocumentReference-SD_BHJXBDQK-s3://kf-study-us...,Aligned Reads Index,crai


For these 38 files
Nothing tells us what the different bam files are.
Nothing tells us which crai applies to which cram.

Perhaps looking at samples for this subject might tell us that sequencing was done on more than one sample, e.g. tumor and somatic.

How do we query for samples. We've seen that there are Specimen resources.

The following is a guess - based on what worked for Documents

In [67]:
response = sess.get(f"{FHIR_SERVER}/Specimen?subject=697194")
bundle = response.json()

specimens = [entry['resource'] for entry in bundle['entry']]
print("# of specimens:{}".format(len(specimens)))

# of specimens:17


OK! Let's look at the specimens

In [66]:
print (len(specimens))
for s in specimens:
    print(json.dumps(s, indent=3))
    print ('_'*50)

17
{
   "resourceType": "Specimen",
   "id": "763287",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2021-06-16T02:52:29.532+00:00",
      "source": "#AOCDAxfya2UvC0DR",
      "profile": [
         "http://hl7.org/fhir/StructureDefinition/Specimen"
      ]
   },
   "identifier": [
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/biospecimens/",
         "value": "BS_PEFRDKDZ"
      },
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/biospecimens/",
         "value": "?study_id=SD_BHJXBDQK&external_aliquot_id=739813"
      },
      {
         "system": "urn:kids-first:unique-string",
         "value": "Specimen-SD_BHJXBDQK-739813"
      }
   ],
   "status": "available",
   "type": {
      "coding": [
         {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0487",
            "code": "TISS",
            "display": "Tissue"
         }
      ],
      "text": "Solid Tissue"
   },
   "subject": {
      "reference": 

It's hard to see what's going on there. Let's see if we can summarize. First it looks like there may have been specimens collected at more than one time. There's an extension used to record age. It's done in a complicated way - by defining and event birth, a relationship "after" and an offset a value and unit. (Wouldn't a CDE for days after birth have been easier?)

Let's look at when the specimens were collected.

In [65]:
for s in specimens:
    if '_collectedDateTime' in s['collection']:
        offsetDuration = s['collection']['_collectedDateTime']['extension'][0]['extension'][2]['valueDuration']
        print("Specimen {} collected at {} {}".format(s['id'], offsetDuration['value'],offsetDuration['unit']))
    else:
        print ("Specimen {} collected at unknown time".format(s['id']))
    

Specimen 763287 collected at 5407 days
Specimen 763285 collected at unknown time
Specimen 763266 collected at 5407 days
Specimen 763261 collected at 5407 days
Specimen 719528 collected at 5407 days
Specimen 719526 collected at 5407 days
Specimen 719525 collected at 5407 days
Specimen 719524 collected at 5407 days
Specimen 719522 collected at 5407 days
Specimen 719521 collected at 5407 days
Specimen 719519 collected at 5407 days
Specimen 719516 collected at 5407 days
Specimen 719514 collected at 5407 days
Specimen 719513 collected at 5407 days
Specimen 719510 collected at 5407 days
Specimen 719508 collected at 5407 days
Specimen 719507 collected at 5407 days


Apart from one unknown specimen all were collected on the same day.

Note that not all specimens are listed

What was collected?

That still doesn't tell us a lot. What was different about the solid tissue samples. Tumor? Normal? And the multiple blood samples can't be distinguished. There's also nothing linking the files to the specimens. Which were used for the sequencing and prodcued the bam files? 

The Collection method and site. The blood may have been collected during the surgical procedure

In [70]:
specimenDetails = []
for specimen in specimens:
    kfid = [did for did in specimen['identifier'] if did['system']=='urn:kids-first:unique-string']
    specimenDetails.append ({
    "id": specimen['id'],
    "urn:kids-first:unique-string": kfid[0]['value'],
    "type": specimen['type']['text'],
    "site": specimen['collection']['bodySite']['text'],
    "method": specimen['collection']['method']['text']
    })
specimensDF = pd.DataFrame(specimenDetails)
specimensDF = specimensDF.sort_values(["urn:kids-first:unique-string"])
specimensDF



,id,urn:kids-first:unique-string,type,site,method
5,719526,Specimen-SD_BHJXBDQK-551153,Solid Tissue,Frontal Lobe,Surgical Resections
4,719528,Specimen-SD_BHJXBDQK-551154,Solid Tissue,Frontal Lobe,Surgical Resections
13,719513,Specimen-SD_BHJXBDQK-668381,Solid Tissue,Frontal Lobe,Surgical Resections
6,719525,Specimen-SD_BHJXBDQK-668389,Solid Tissue,Frontal Lobe,Surgical Resections
16,719507,Specimen-SD_BHJXBDQK-683272,Peripheral Whole Blood,Frontal Lobe,Surgical Resections
10,719519,Specimen-SD_BHJXBDQK-683273,Peripheral Whole Blood,Frontal Lobe,Surgical Resections
15,719508,Specimen-SD_BHJXBDQK-683274,Peripheral Whole Blood,Frontal Lobe,Surgical Resections
7,719524,Specimen-SD_BHJXBDQK-683275,Peripheral Whole Blood,Frontal Lobe,Surgical Resections
9,719521,Specimen-SD_BHJXBDQK-683373,Peripheral Whole Blood,Frontal Lobe,Surgical Resections
14,719510,Specimen-SD_BHJXBDQK-689897,Solid Tissue,Frontal Lobe,Surgical Resections


### Back to Genomics
Which is our goal, and the leukemia study

Find the subjects in the study

In [85]:
studyid = 577137
response = sess.get(f"{FHIR_SERVER}/ResearchSubject?study={studyid}")
bundle = response.json()

subjects = [entry['resource'] for entry in bundle['entry']]
print("# of subjects:{}".format(len(subjects)))

print("# of subjects in study {}:{}".format(studyid, len(subjects)))
for s in subjects[30:40]:
    print(json.dumps(s, indent=3))
    print('_'*50)

# of subjects:50
# of subjects in study 577137:50
{
   "resourceType": "ResearchSubject",
   "id": "577727",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2021-04-28T23:57:48.134+00:00",
      "source": "#H67MmCmsmuBCPPdj",
      "profile": [
         "http://hl7.org/fhir/StructureDefinition/ResearchSubject"
      ]
   },
   "identifier": [
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/participants/",
         "value": "PT_239J4883"
      },
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/participants/",
         "value": "?study_id=SD_W0V965XZ&external_id=SJBALL056715"
      },
      {
         "system": "urn:kids-first:unique-string",
         "value": "ResearchSubject|SD_W0V965XZ|SJBALL056715"
      }
   ],
   "status": "off-study",
   "study": {
      "reference": "ResearchStudy/577137"
   },
   "individual": {
      "reference": "Patient/576435"
   }
}
__________________________________________________
{
   "resourceTy

Define a function to get Files

In [86]:
def getFiles(sess, subject_id):
    response = sess.get(f"{FHIR_SERVER}/DocumentReference?subject={subject_id}")
    bundle = response.json()
    documents = [entry['resource'] for entry in bundle['entry']]

    print("# of documents for subject {} :{}".format(subject_id, len(documents)))

    myDocs = []
    for d in documents:
        #print(json.dumps(d, indent=3))
        myDocs.append({

        "type":d['type']['text'],
        "format":d['content'][0]['format']['display']
        })

    docsDF = pd.DataFrame(myDocs)
    docsDF = docsDF.sort_values(["type", "format"])
    print(docsDF)

And use that function to get the files for the patients above

In [88]:
for s in subjects[30:40]:
    patient = s['individual']['reference']
    getFiles(sess, patient)
    print('_'*50)

# of documents for subject Patient/576435 :8
                  type format
7        Aligned Reads    bam
0        Aligned Reads   cram
2  Aligned Reads Index   crai
1      Unaligned Reads  fastq
3      Unaligned Reads  fastq
5        Variant Calls    vcf
6                 gVCF   gvcf
4           gVCF Index    tbi
__________________________________________________
# of documents for subject Patient/576439 :8
                  type format
4        Aligned Reads    bam
0        Aligned Reads   cram
1  Aligned Reads Index   crai
3      Unaligned Reads  fastq
6      Unaligned Reads  fastq
7        Variant Calls    vcf
2                 gVCF   gvcf
5           gVCF Index    tbi
__________________________________________________
# of documents for subject Patient/576438 :10
                  type format
5        Aligned Reads    bam
9        Aligned Reads   cram
0  Aligned Reads Index   crai
2      Unaligned Reads  fastq
4      Unaligned Reads  fastq
6      Unaligned Reads  fastq
8      Unali

### Can we search for bams specifically?

#TODO fix this

In [116]:
response = sess.get(f"{FHIR_SERVER}/DocumentReference?format__text=bam")
bundle = response.json()

bams = [entry['resource'] for entry in bundle['entry']]
print("# of bams:{}".format(len(bams)))

import pandas as pd

file1 = open("bam_subjects.txt","w") 
for d in bams:
    #print(djson['subject'])
    file1.write(d['subject']['reference']+'\n')

file1.close()

KeyError: 'entry'

Try getting everything and inspect the results

In [118]:
response = sess.get(f"{FHIR_SERVER}/DocumentReference")
bundle = response.json()

documents = [entry['resource'] for entry in bundle['entry']]
print("# of documents:{}".format(len(documents)))

#import pandas as pd

file1 = open("all_docs.txt","w") 
for d in documents:
    if 'format' in d['content'][0]:
        fmt = d['content'][0]['format']['display']
    else:
        fmt = 'none'
    file1.write("{}\t{}\n".format(fmt,d['subject']['reference']))                
file1.close()



# of documents:50


Try a different study

In [89]:
studyid = 701322
response = sess.get(f"{FHIR_SERVER}/ResearchSubject?study={studyid}")
bundle = response.json()

subjects = [entry['resource'] for entry in bundle['entry']]
print("# of subjects:{}".format(len(subjects)))

print("# of subjects in study {}:{}".format(studyid, len(subjects)))
for s in subjects[30:40]:
    print(json.dumps(s, indent=3))
    print('_'*50)

# of subjects:50
# of subjects in study 701322:50
{
   "resourceType": "ResearchSubject",
   "id": "705462",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2021-06-16T02:04:43.838+00:00",
      "source": "#Djd3HyXDFStZyuOw",
      "profile": [
         "http://hl7.org/fhir/StructureDefinition/ResearchSubject"
      ]
   },
   "identifier": [
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/participants/",
         "value": "PT_41TVK406"
      },
      {
         "system": "https://kf-api-dataservice.kidsfirstdrc.org/participants/",
         "value": "?study_id=SD_BHJXBDQK&external_id=C3291480"
      },
      {
         "system": "urn:kids-first:unique-string",
         "value": "ResearchSubject-SD_BHJXBDQK-C3291480"
      }
   ],
   "status": "off-study",
   "study": {
      "reference": "ResearchStudy/701322"
   },
   "individual": {
      "reference": "Patient/701278"
   }
}
__________________________________________________
{
   "resourceType": "Re

In [113]:
#TODO - fix the query by format below

In [111]:
def getFiles2(sess, subject_id, file_type):
    response = sess.get(f"{FHIR_SERVER}/DocumentReference?subject={subject_id}&format_display={file_type}")
    bundle = response.json()
    myDocs = []
    if 'entry' in bundle:
        documents = [entry['resource'] for entry in bundle['entry']]

        print("# of {} files for subject {} :{}".format(file_type, subject_id, len(documents)))

        
        for d in documents:
            #print(json.dumps(d, indent=3))
            myDocs.append({

            "type":d['type']['text'],
            "format":d['content'][0]['format']['display']
            })

    if len(myDocs) > 0:
        docsDF = pd.DataFrame(myDocs)
        docsDF = docsDF.sort_values(["type", "format"])
        print(docsDF)
    else:
        print("No {} files found for subject {}".format(file_type, subject_id))

In [112]:
for s in subjects[30:40]:
    patient = s['individual']['reference']
    getFiles2(sess, patient, 'bam')
    print('_'*50)

No bam files found for subject Patient/701278
__________________________________________________
No bam files found for subject Patient/701284
__________________________________________________
No bam files found for subject Patient/701297
__________________________________________________
No bam files found for subject Patient/701272
__________________________________________________
No bam files found for subject Patient/701290
__________________________________________________
No bam files found for subject Patient/701280
__________________________________________________
No bam files found for subject Patient/701279
__________________________________________________
No bam files found for subject Patient/701319
__________________________________________________
No bam files found for subject Patient/701271
__________________________________________________
No bam files found for subject Patient/701292
__________________________________________________
